In [45]:
import pandas as pd
import numpy as np
week1_raw = pd.read_csv("week1_results.csv")
#week2_raw = pd.read_csv("week2_results.csv)
#week1_raw.head()


In [46]:
#function to combine picks and point values
def combine(df1):
    my_file = df1
    rows = my_file.shape[0]
    i = 2 
    while i < rows:
        for col in my_file.loc[:, my_file.columns != 'Name']:
            my_file[col][i+1] = float(my_file[col][i+1])
            my_file[col][i] = [my_file[col][i], my_file[col][i+1]]
        i = i+2
    return my_file

week1 = combine(week1_raw)

In [ ]:
#drop unneeded rows
week1.drop([5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 42, 43], axis=0, inplace=True)
week1.tail()

#i = 3
#rows = week1.shape[1]
#while i < 6:
#    print(week1.iloc[])
#    i = i + 2

In [ ]:
#add spread for home team
#for col in week1.columns:
#    week1[col][1] = [week1[col][1], -week1[col][2][1]]


In [41]:
#function to calculate total points
i = 4
week1['Score'] = 0
while i < 42:
    #print(i)
    for col in week1.loc[:, ~week1.columns.isin(['Name', 'Score'])]:
        if week1[col][0] == week1[col][i][0]:
            week1['Score'][i] = week1['Score'][i] + week1[col][i][1]
            #print(week1['Name'][i], ' the ', col, 'was a good game')
        #else:
            #print(week1['Name'][i], "haha, you suck")
    i = i + 2

C:\Users\mtzwa\AppData\Local\Temp\ipykernel_4676\1905780758.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  week1['Score'][i] = week1['Score'][i] + week1[col][i][1]


In [43]:
#week1.sort_values(week1['Score'])
#week1[['Name', 'Score']]
week1.head()

,Name,Game 1,Game 2,Game 3,Game 4,Game 5,Game 6,Game 7,Game 8,Game 9,Game 10,Game 11,Game 12,Game 13,Game 14,Game 15,Game 16,Score
0,Winner,BUF,NO,BAL,PIT,MIA,WAS,CHI,CLE,TIE,PHI,LAC,MIN,NYG,KC,TB,SEA,0
1,Home team,LAR,ATL,NYJ,CIN,MIA,WAS,CHI,CAR,HOU,DET,LAC,MIN,TEN,ARI,DAL,SEA,0
2,Away Team,"[BUF, -2.0]","[NO, -5.5]","[BAL, -7.0]","[PIT, 6.5]","[NE, 3.5]","[JAX, 2.5]","[SF, -7.0]","[CLE, 1.5]","[IND, -7.0]","[PHI, -4.0]","[LV, 3.5]","[GB, -1.5]","[NYG, 5.5]","[KC, -6.0]","[TB, -2.5]","[DEN, -6.5]",0
3,Spread,-2.0,-5.5,-7.0,6.5,3.5,2.5,-7.0,1.5,-7.0,-4.0,3.5,-1.5,5.5,-6.0,-2.5,-6.5,0
4,abfarias,"[BUF, 1.0]","[ATL, 16.0]","[NYJ, 15.0]","[CIN, 14.0]","[MIA, 13.0]","[WAS, 12.0]","[CHI, 11.0]","[CAR, 10.0]","[HOU, 9.0]","[DET, 8.0]","[LAC, 7.0]","[MIN, 6.0]","[TEN, 5.0]","[ARI, 4.0]","[DAL, 3.0]","[SEA, 2.0]",52


In [73]:
#upset count

def upset_counter(df):
    upset_count = 0
    fav_count = 0
    tie_count = 0
    for col in week1.loc[:, ~week1.columns.isin(['Name', 'Score'])]:
        if week1[col][3] < 0 and week1[col][1] == week1[col][0]: #if away team is favorite and home team won
            upset_count = upset_count + 1
            #print('Home dog pulling it off: ', week1[col][1])
        elif week1[col][3] > 0 and week1[col][2][0] == week1[col][0]: #if home team is favorite and away team won
            upset_count = upset_count + 1
            #print('Road upset alert: ', week1[col][2][0])
        elif week1[col][0] == "TIE":
            tie_count = tie_count + 1
            #print("A friggen tie...  ", week1[col][2])
        else:
            fav_count = fav_count + 1
    
    return upset_count, fav_count, tie_count

In [76]:
upset_count = upset_counter(week1)[0]
fav_count = upset_counter(week1)[1]
tie_count = upset_counter(week1)[2]

print("There were ", upset_count, " upsets in week 1.")



There were  6  upsets in week 1.


In [38]:
#upset_score = 0
#for col in week1.columns:
    #if week1[col][1][1] == week1[0]:
    #    upset_score = upset_score + week1[col][1][1]
    #else if week1[0] == week1[col][1][1]
